In [1]:
from radicalbit_platform_sdk.client import Client
from radicalbit_platform_sdk.models import (
    AwsCredentials,
    CreateModel,
    DataType,
    ModelType,
    ColumnDefinition,
    OutputType,
    Granularity,
)

from datetime import datetime 
import pandas as pd


In [2]:
# Create the Client
base_url = "http://localhost:9000"
client = Client(base_url)

In [3]:
reference = pd.read_csv("./data/binary_classification/reference-income.csv")
reference.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K,pred_id,prob_class_1,prediction,timestamp
0,37,Private,174844,Some-college,10,Married-civ-spouse,Other-service,Husband,White,Male,0,0,17,United-States,0,fe2f8443-b644-4ba5-a130-45336a0a04d6,0.179987,0,2024-01-09 23:59:00
1,42,NaN,167678,11th,7,Married-civ-spouse,NaN,Husband,White,Male,0,0,22,Ecuador,0,b85abaca-581e-4988-8392-03460b3aa5d5,0.126728,0,2024-01-10 00:08:00
2,27,Self-emp-not-inc,115438,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,Ireland,1,01d05d15-8d34-4451-97d5-99dff5ee9217,0.248176,0,2024-01-10 00:21:00
3,32,Private,295589,Masters,14,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,1977,40,United-States,1,01a9168f-fbb9-46e4-acfd-c1be174a9373,0.853719,1,2024-01-10 00:33:00
4,20,Private,107746,11th,7,Never-married,Transport-moving,Other-relative,White,Male,0,0,40,Guatemala,0,3a6074d2-b42f-4853-aeac-d164d36baad2,0.012086,0,2024-01-10 00:42:00


In [4]:
# Create the Model
model = CreateModel(
    name=f"Linear-Regressor-{str(datetime.now()).replace(' ', '-').replace(':', '-').split('.')[0]}",
    modelType=ModelType.BINARY,
    dataType=DataType.TABULAR,
    granularity=Granularity.HOUR,
    features=[
        ColumnDefinition(name="age", type="int",field_type="numerical"),
        ColumnDefinition(name="workclass", type="string",field_type="categorical"),
        ColumnDefinition(name="fnlwgt", type="int",field_type="numerical"),
        ColumnDefinition(name="education", type="string",field_type="categorical"),
        ColumnDefinition(name="educational-num", type="int",field_type="numerical"),
        ColumnDefinition(name="marital-status", type="string",field_type="categorical"),
        ColumnDefinition(name="occupation", type="string",field_type="categorical"),
        ColumnDefinition(name="relationship", type="string",field_type="categorical"),
        ColumnDefinition(name="race", type="string",field_type="categorical"),
        ColumnDefinition(name="gender", type="string",field_type="categorical"),
        ColumnDefinition(name="capital-gain", type="int",field_type="numerical"),
        ColumnDefinition(name="capital-loss", type="int",field_type="numerical"),
        ColumnDefinition(name="hours-per-week", type="int",field_type="numerical"),
        ColumnDefinition(name="native-country", type="string",field_type="categorical"),
    ],
    outputs=OutputType(
        prediction=ColumnDefinition(name="prediction", type="int",field_type="numerical"),
        output=[
            ColumnDefinition(name="prob_class_1", type="float",field_type="numerical"),
            ColumnDefinition(name="prediction", type="int",field_type="numerical")
        ],
    ),
    target=ColumnDefinition(name="income_>50K", type="int",field_type="numerical"),
    timestamp=ColumnDefinition(name="timestamp", type="datetime",field_type="datetime"),
)

model = client.create_model(model)
print(model)
print(model.name())
print(model.uuid())

Linear-Regressor-2024-12-16-11-44-50
b1c5a434-0d57-4fb0-a528-70dc2386d47c


In [5]:
# load the reference dataset
ref = model.load_reference_dataset(
    file_name="./data/binary_classification/reference-income.csv",
    bucket="test-bucket",
    aws_credentials=AwsCredentials(
        access_key_id="minio",
       secret_access_key="minio123",
       default_region="us-east-1",
        endpoint_url="http://localhost:9090"
    )
)

print(ref)

In [6]:
# load the current dataset

if ref.status().value == 'SUCCEEDED' :
    cur1 = model.load_current_dataset(
        file_name="./data/binary_classification/current1-income.csv",
        correlation_id_column="pred_id",
        bucket="test-bucket",
        aws_credentials=AwsCredentials(
            access_key_id="minio",
           secret_access_key="minio123",
           default_region="us-east-1",
            endpoint_url="http://localhost:9090"
        )
    )
else:
    print("PIPPO")

PIPPO


In [7]:
from radicalbit_platform_sdk.charts import RadicalbitChart,RbitDistributionData

RadicalbitChart().DistributionChart(data=RbitDistributionData(
    model=model,
    reference_data=ref.data_quality()
))






ValidationError: 1 validation error for RbitDistributionData
model
  Input should be a valid dictionary or instance of BaseModelDefinition [type=model_type, input_value=<radicalbit_platform_sdk....l object at 0x10d579c50>, input_type=Model]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type